In [125]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
project_dir = "/mnt/c/SKH/ai_lab_13/projects/nlp-text-summarization/song"
import os
import sys
sys.path.append(
    project_dir
)
from src.utils.utils import *

import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

train_df = pd.read_csv(os.path.join(project_dir, 'data','train_clean_special_tokens.csv'))
val_df = pd.read_csv(os.path.join(project_dir, 'data','dev.csv'))
test_df = pd.read_csv(os.path.join(project_dir, 'data','test.csv'))
sub_df = pd.read_csv(os.path.join(project_dir, 'data','submission_08020735.csv'))

test_df['summary'] = sub_df['summary']

## special token 확인

In [80]:
special_tokens = [
    '#Person1#', '#Person2#', '#Person3#', '#Person4#', 
    '#Person5#', '#Person6#', '#Person7#', 
    '#PhoneNumber#',
    '#Address#',
    '#DateOfBirth#',
    '#PassportNumber#',
    '#SSN#',
    '#CardNumber#',
    '#CarNumber#',
    '#Email#',
]

In [4]:
tds = train_df[['fname','dialogue']]
tds[special_tokens] = 0
def include_special_token(row):
    for st in special_tokens:
        if st in row['dialogue']:
            row[st] = 1
    return row
from tqdm import tqdm
tqdm.pandas()
tds = tds.progress_apply(include_special_token, axis=1)

/tmp/ipykernel_899/199812826.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tds[special_tokens] = 0
100%|██████████████████████████████████████████████████████████████████████████| 12457/12457 [00:01<00:00, 10516.55it/s]


In [5]:
tds[special_tokens].sum()

#Person1#           12457
#Person2#           12457
#Person3#             122
#Person4#              15
#Person5#               5
#Person6#               2
#Person7#               1
#PhoneNumber#         151
#Address#              81
#DateOfBirth#          20
#PassportNumber#        5
#SSN#                   3
#CardNumber#           12
#CarNumber#             7
#Email#                13
dtype: int64

- 화자 : 총 7명까지 존재

In [6]:
vds = val_df[['fname','dialogue']]
vds[special_tokens] = 0
def include_special_token(row):
    for st in special_tokens:
        if st in row['dialogue']:
            row[st] = 1
    return row
vds = vds.progress_apply(include_special_token, axis=1)
vds[special_tokens].sum()

/tmp/ipykernel_899/2116243011.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vds[special_tokens] = 0
100%|██████████████████████████████████████████████████████████████████████████████| 499/499 [00:00<00:00, 10442.29it/s]


#Person1#           499
#Person2#           499
#Person3#             5
#Person4#             1
#Person5#             0
#Person6#             0
#Person7#             0
#PhoneNumber#         5
#Address#             2
#DateOfBirth#         2
#PassportNumber#      0
#SSN#                 0
#CardNumber#          0
#CarNumber#           0
#Email#               0
dtype: int64

In [7]:
ttds = test_df[['fname','dialogue']]
ttds[special_tokens] = 0
def include_special_token(row):
    for st in special_tokens:
        if st in row['dialogue']:
            row[st] = 1
    return row
ttds = ttds.progress_apply(include_special_token, axis=1)
ttds[special_tokens].sum()

100%|███████████████████████████████████████████████████████████████████████████████| 499/499 [00:00<00:00, 6513.12it/s]


#Person1#           499
#Person2#           499
#Person3#             4
#Person4#             0
#Person5#             0
#Person6#             0
#Person7#             0
#PhoneNumber#         5
#Address#             1
#DateOfBirth#         0
#PassportNumber#      1
#SSN#                 0
#CardNumber#          0
#CarNumber#           0
#Email#               0
dtype: int64

#### special token은 항상 요약문에 포함되는 가?

In [8]:
# person은 제외하고 분석
special_tokens_noperson = list(set(special_tokens) - set({'#Person1#', '#Person2#', '#Person3#', '#Person4#', 
    '#Person5#', '#Person6#', '#Person7#'}))
special_tokens_onlyperson = ['#Person1#', '#Person2#', '#Person3#', '#Person4#', 
    '#Person5#', '#Person6#', '#Person7#']

In [4]:
from typing import List, Set

def find_mismatched_special_tokens(
    train_df: pd.DataFrame,
    special_tokens_list: List[str]
) -> List[str]:
    """
    train_df에서 dialogue에 special_tokens_noperson 중 특정 토큰이 포함되었지만
    summary에는 포함되지 않은 fname을 찾아 리스트로 반환합니다.

    Args:
        train_df (pd.DataFrame): 'fname', 'dialogue', 'summary' 컬럼을 포함하는 DataFrame.
        special_tokens_noperson (List[str]): 확인하고자 하는 특별 토큰 리스트.

    Returns:
        List[str]: dialogue에는 포함되었지만 summary에는 해당 special_token이 없는 fname 리스트.
    """
    
    # 중복 확인을 위해 special_tokens_noperson을 Set으로 변환
    special_tokens_set = set(special_tokens_list)
    
    mismatched_fnames = []

    # 데이터프레임의 각 행을 순회합니다.
    for index, row in train_df.iterrows():
        fname = row['fname']
        dialogue = str(row['dialogue']) # NaN 값이나 다른 타입이 있을 수 있으므로 문자열로 변환
        summary = str(row['summary'])   # NaN 값이나 다른 타입이 있을 수 있으므로 문자열로 변환

        # 1. dialogue에 special_tokens_noperson 중 어느 것이 포함되어 있는지 확인
        dialogue_contained_tokens = {
            token for token in special_tokens_set if token in dialogue
        }

        # dialogue에 special_tokens_noperson이 하나도 포함되어 있지 않다면 건너뜁니다.
        if not dialogue_contained_tokens:
            continue

        # 2. dialogue에 포함된 special_tokens_noperson이 모두 summary에도 포함되어 있는지 확인
        # dialogue에 포함된 토큰 중 summary에 없는 토큰을 찾습니다.
        missing_in_summary = {
            token for token in dialogue_contained_tokens if token not in summary
        }

        # 3. 만약 dialogue에는 포함되었지만 summary에 포함되지 않은 special_tokens_noperson이 있다면
        #    해당 fname을 따로 리스트에 저장.
        if missing_in_summary:
            mismatched_fnames.append(fname)
            # (선택 사항) 어떤 토큰이 누락되었는지 출력하여 디버깅에 도움을 줄 수 있습니다.
            # print(f"fname: {fname}, Missing tokens in summary: {missing_in_summary}")

    return mismatched_fnames

def find_special_tokens(df, fnames, special_tokens_list):
    filtered_df = df[df['fname'].isin(fnames)].copy()
    special_tokens_set = set(special_tokens_list)
    # 각 텍스트에서 스페셜 토큰을 추출하는 헬퍼 함수
    def extract_tokens_from_text(text: str, tokens_set: Set[str]) -> List[str]:
        if not isinstance(text, str): # NaN 값 처리
            return []
        
        extracted = []
        for token in tokens_set:
            if token in text:
                extracted.append(token)
        return sorted(list(set(extracted))) # 중복 제거 및 정렬하여 일관성 유지
    # apply() 함수를 사용하여 새로운 컬럼을 생성합니다.
    filtered_df['dialogue_st'] = filtered_df['dialogue'].apply(
        lambda x: extract_tokens_from_text(x, special_tokens_set)
    )
    filtered_df['summary_st'] = filtered_df['summary'].apply(
        lambda x: extract_tokens_from_text(x, special_tokens_set)
    )
    
    return filtered_df

In [10]:
train_mismatched_fnames = find_mismatched_special_tokens(train_df, special_tokens)
train_mismatched_filtered_df = find_special_tokens(train_df, train_mismatched_fnames, special_tokens)
train_mismatched_filtered_df['dialogue_st_len'] = train_mismatched_filtered_df['dialogue_st'].apply(len)
train_mismatched_filtered_df['summary_st_len'] = train_mismatched_filtered_df['summary_st'].apply(len)
print("mismatch 총 개수:", len(train_mismatched_fnames))
display(train_mismatched_filtered_df.head(3))
print(train_mismatched_filtered_df[['dialogue_st_len','summary_st_len']].describe())

mismatch 총 개수: 4402


,fname,dialogue,summary,topic,dialogue_st,summary_st,dialogue_st_len,summary_st_len
0,train_0,"#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다...","Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필...",건강검진,"[#Person1#, #Person2#]",[],2,0
1,train_1,"#Person1#: 안녕하세요, Mrs. Parker. 잘 지내셨나요?\n#Pers...","Mrs. Parker가 Ricky와 함께 백신 접종을 위해 방문하였고, Dr. Pe...",백신 접종,"[#Person1#, #Person2#]",[],2,0
4,train_4,"#Person1#: 안녕, 오늘 너무 멋져 보이네요. 저랑 춤 한 곡 추실래요?\n...","Malik은 Wen과 Nikki에게 춤을 제안하고, Wen은 발을 밟는 것을 감수하...",춤 제안,"[#Person1#, #Person2#]",[],2,0


       dialogue_st_len  summary_st_len
count      4402.000000     4402.000000
mean          2.089050        0.630850
std           0.343437        0.546233
min           2.000000        0.000000
25%           2.000000        0.000000
50%           2.000000        1.000000
75%           2.000000        1.000000
max           7.000000        6.000000


In [11]:
val_mismatched_fnames = find_mismatched_special_tokens(val_df, special_tokens_noperson)
val_mismatched_filtered_df = find_special_tokens(val_df, val_mismatched_fnames, special_tokens)
val_mismatched_filtered_df['dialogue_st_len'] = val_mismatched_filtered_df['dialogue_st'].apply(len)
val_mismatched_filtered_df['summary_st_len'] = val_mismatched_filtered_df['summary_st'].apply(len)
print("mismatch 총 개수:", len(val_mismatched_fnames))
display(val_mismatched_filtered_df.head(3))
print(val_mismatched_filtered_df[['dialogue_st_len','summary_st_len']].describe())

mismatch 총 개수: 9


,fname,dialogue,summary,topic,dialogue_st,summary_st,dialogue_st_len,summary_st_len
38,dev_38,"#Person1#: Sound and Vision, 행정부입니다. Eve Wheel...","John Cruise는 Eve에게 비디오 녹화기 문제로 불만을 제기하고, 기기가 테...",비디오 녹화기 불만 제기,"[#Person1#, #Person2#, #PhoneNumber#]",[],3,0
118,dev_118,#Person1#: 여기는 911입니다. \n#Person2#: 빨리 구급차 보내주...,#Person2#가 남편의 심장마비로 911에 전화하여 #Person1#에게 구급차...,응급 이송 요청,"[#Address#, #Person1#, #Person2#]","[#Person1#, #Person2#]",3,2
123,dev_123,#Person1#: 이 직무에 관심 가져주셔서 감사합니다. 저희 쪽에서 연락드리겠습...,#Person2#가 면접을 마치고 결과를 통보받기 위해 #Person1#에게 본인의...,면접 후 연락,"[#Person1#, #Person2#, #PhoneNumber#]","[#Person1#, #Person2#]",3,2


       dialogue_st_len  summary_st_len
count              9.0        9.000000
mean               3.0        1.111111
std                0.0        0.927961
min                3.0        0.000000
25%                3.0        0.000000
50%                3.0        1.000000
75%                3.0        2.000000
max                3.0        2.000000


In [12]:
val_mismatched_filtered_df

,fname,dialogue,summary,topic,dialogue_st,summary_st,dialogue_st_len,summary_st_len
38,dev_38,"#Person1#: Sound and Vision, 행정부입니다. Eve Wheel...","John Cruise는 Eve에게 비디오 녹화기 문제로 불만을 제기하고, 기기가 테...",비디오 녹화기 불만 제기,"[#Person1#, #Person2#, #PhoneNumber#]",[],3,0
118,dev_118,#Person1#: 여기는 911입니다. \n#Person2#: 빨리 구급차 보내주...,#Person2#가 남편의 심장마비로 911에 전화하여 #Person1#에게 구급차...,응급 이송 요청,"[#Address#, #Person1#, #Person2#]","[#Person1#, #Person2#]",3,2
123,dev_123,#Person1#: 이 직무에 관심 가져주셔서 감사합니다. 저희 쪽에서 연락드리겠습...,#Person2#가 면접을 마치고 결과를 통보받기 위해 #Person1#에게 본인의...,면접 후 연락,"[#Person1#, #Person2#, #PhoneNumber#]","[#Person1#, #Person2#]",3,2
136,dev_136,"#Person1#: 여보세요. \n#Person2#: 여보세요, 아빠? \n#Per...",Mary가 아버지에게 가족의 안부를 물으며 통화한다.,가족 안부 통화,"[#DateOfBirth#, #Person1#, #Person2#]",[],3,0
151,dev_151,"#Person1#: 안녕하세요, 호텔 방을 예약하고 싶어요. \n#Person2#:...",Michelle은 John Sandals 씨가 뉴욕을 방문하는 4월 14일부터 4월...,호텔 방 예약,"[#Person1#, #Person2#, #PhoneNumber#]",[],3,0
268,dev_268,"#Person1#: 네, 인터뷰는 여기까지입니다. 이 직업에 관심 가져주셔서 감사합...","인터뷰가 종료되었고, #Person1#은 #Person2#에게 최종 결정을 알리기 ...",구직 인터뷰,"[#Person1#, #Person2#, #PhoneNumber#]","[#Person1#, #Person2#]",3,2
331,dev_331,"#Person1#: 안녕하세요, 제 이름은 David Parker예요. 신문에 광고...","David Parker가 아파트 광고와 관련된 위치, 시설, 가격에 대해 문의하면서...",아파트 임대,"[#Address#, #Person1#, #Person2#]",[#Person2#],3,1
397,dev_397,"#Person1#: 안녕하세요, 예약 데스크입니다. 무엇을 도와드릴까요?\n#Per...",#Person1#은 Nancy의 스탠다드룸 예약을 처리하면서 그녀의 개인 정보를 요...,호텔 방 예약,"[#Person1#, #Person2#, #PhoneNumber#]",[#Person1#],3,1
469,dev_469,"#Person1#: 언제 태어나셨어요, 물어봐도 될까요?\n#Person2#: #D...","#Person1#은 #Person2#에게 생일과 영화 시작 시간을 묻고, 영화 시간...",생일과 영화 시간,"[#DateOfBirth#, #Person1#, #Person2#]","[#Person1#, #Person2#]",3,2


In [16]:
test_mismatched_fnames = find_mismatched_special_tokens(test_df, special_tokens_noperson)
test_mismatched_filtered_df = find_special_tokens(test_df, test_mismatched_fnames, special_tokens)
test_mismatched_filtered_df['dialogue_st_len'] = test_mismatched_filtered_df['dialogue_st'].apply(len)
test_mismatched_filtered_df['summary_st_len'] = test_mismatched_filtered_df['summary_st'].apply(len)
print("mismatch 총 개수:", len(test_mismatched_fnames))
display(test_mismatched_filtered_df.head(3))
print(test_mismatched_filtered_df[['dialogue_st_len','summary_st_len']].describe())

mismatch 총 개수: 7


,fname,dialogue,summary,dialogue_st,summary_st,dialogue_st_len,summary_st_len
54,test_54,"#Person1#: 여보세요, #PhoneNumber#입니다. \n#Person2#...",데이브 톰슨이 샐리에게 전화를 걸어 짐이 아직 돌아오지 않았다고 알립니다. ...,"[#Person1#, #Person2#, #PhoneNumber#]",[],3,0
146,test_146,#Person1#: 혹시 다른 지역도 추천해줄 수 있어요?\n#Person2#: 네...,"#Person1# 은 #Person2# 에게 웨스트 더비, 플랫, 그리고 John...","[#Person1#, #Person2#, #PhoneNumber#]","[#Person1#, #Person2#]",3,2
148,test_148,"#Person1#: 안녕하세요, Dan. 9월 10일에 100대 컴퓨터 주문 건에 ...",Dan은 9월 10일에 100대 컴퓨터 주문 건에 대해 확인하려고 전화했으나 인...,"[#Person1#, #Person2#, #PhoneNumber#]",[#Person1#],3,1


       dialogue_st_len  summary_st_len
count              7.0        7.000000
mean               3.0        1.142857
std                0.0        0.899735
min                3.0        0.000000
25%                3.0        0.500000
50%                3.0        1.000000
75%                3.0        2.000000
max                3.0        2.000000


In [10]:
additional_special_tokens = ["#FlightNumber#","#PersonName#","#Name#","#Alex#","#Bob#","#Kristin#","#Liliana#","#Mike#"]
def find_additional_special_tokens(row, test=False):
    dialogue = row['dialogue']
    summary = ""
    topic = ""
    if not test:
        summary = row['summary']
        topic = row['topic']
    for t in [dialogue,summary,topic]:
        for ast in additional_special_tokens:
            if ast in t:
                row['st'] = 1
                return row
    return row

In [44]:
from tqdm import tqdm
tqdm.pandas()

train_df['st'] = 0
train_df = train_df.progress_apply(find_additional_special_tokens, axis=1)

100%|██████████████████████████████████████████████████████████████████████████| 12457/12457 [00:00<00:00, 27714.10it/s]


In [45]:
print(train_df['st'].sum())
train_df[train_df['st']>=1]

8


,fname,dialogue,summary,topic,st
1946,train_1946,"#Person1#: 본론으로 들어가죠.\n#Person2#: 네, 샘플 보셨나요?\...","한 고객은 속옷 주문을 원하고 가격 협상을 진행하였습니다. 협상 결과, 10,000...",속옷 주문 협상,1
3903,train_3903,"#Person1#: 안녕하세요, 손님. 예약하셨나요?\n#Person2#: 네, 제...",#PersonName#은 테이블 예약을 했습니다. #Person1#은 #Person...,음식점에서 식사,1
5614,train_5614,"#Person1#: 무엇을 도와드릴까요?\n#Person2#: 네, 저녁식사 예약을...",#Person1#이 #PersonName#의 여섯 커플을 위한 프라이빗 다이닝룸 예...,저녁식사 예약,1
6348,train_6348,"#Person1#: 안녕하세요, 손님. 저희 레스토랑에 오신 것을 환영합니다.\n#...",#Name#씨가 예약된 레스토랑에 도착합니다. #Person1#이 그를 환영하며 메...,레스토랑 방문,1
6608,train_6608,"#Person1#: 저 Flight FA2028 비행기를 타려고 하는데, 시간 맞춰...",#Person1#은 #FlightNumber# 비행기를 타려 했지만 제시간에 도착하...,비행기 지연,1
6829,train_6829,"#Person1#: 자, 그럼 내년 우리의 마케팅 전략에 대한 개요를 마치겠습니다....","#Person2#는 발표의 전략 개요에 대해 #Alex#를 칭찬하며, 가격 전략에 ...",발표 칭찬,1
7017,train_7017,#Person1#: 오늘의 주요 뉴스였습니다. 이제 국제 날씨를 #Mike Sand...,#Bob#과 함께한 #Mike Sanderson#이 자세한 국제 날씨 보고를 전달했...,국제 기상 재해,1
11367,train_11369,"#Person1#: 빈방 있어요?\n#Person2#: 네, 6층에 좋은 방이 있어...","#Person2#는 #Person1#에게 5일간 아침 식사가 포함된 방을 예약하고,...",호텔 서비스,1


In [18]:
val_df['st'] = 0
val_df = val_df.progress_apply(find_additional_special_tokens, axis=1)
print(val_df['st'].sum())

100%|██████████████████████████████████████████████████████████████████████████████| 499/499 [00:00<00:00, 28232.47it/s]

0


In [19]:
test_df['st'] = 0
test_df = test_df.progress_apply(lambda x: find_additional_special_tokens(x,test=True), axis=1)
print(test_df['st'].sum())

100%|██████████████████████████████████████████████████████████████████████████████| 499/499 [00:00<00:00, 42986.25it/s]

0


In [54]:
index = 3
print(train_df[train_df['st']>=1].iloc[index]['dialogue'])
print(train_df[train_df['st']>=1].iloc[index]['summary'])
print(train_df[train_df['st']>=1].iloc[index]['fname'])

#Person1#: 안녕하세요, 손님. 저희 레스토랑에 오신 것을 환영합니다.
#Person2#: 감사합니다.
#Person1#: 예약하셨나요?
#Person2#: 네, 했습니다.
#Person1#: 성함을 알 수 있을까요?
#Person2#: 네, #Name#입니다.
#Person1#: #Name#님, 기대하고 있었습니다. 이쪽으로 오세요. 이 자리가 괜찮으신가요?
#Person2#: 아, 이 자리 괜찮습니다.
#Person1#: 앉으세요.
#Person2#: 감사합니다.
#Person1#: 메뉴입니다.
#Name#씨가 예약된 레스토랑에 도착합니다. #Person1#이 그를 환영하며 메뉴를 제공합니다.
train_6348


In [55]:
train_df = train_df[['fname','dialogue','summary','topic']]

In [67]:
# 데이터 정제
import warnings
warnings.filterwarnings('ignore')
row = train_df[train_df['fname']=='train_1946']
row['dialogue'] = row['dialogue'].str.replace("#Name#","#Person1#")
row['summary'] = row['summary'].str.replace("#Name#","#Person1#")
train_df[train_df['fname']=='train_1946'] = row

row = train_df[train_df['fname']=='train_3903']
row['dialogue'] = row['dialogue'].str.replace("#PersoneName#","#Person2#")
row['summary'] = row['summary'].str.replace("#PersoneName#","#Person2#")
train_df[train_df['fname']=='train_3903'] = row

row = train_df[train_df['fname']=='train_5614']
row['dialogue'] = row['dialogue'].str.replace("#PersoneName#","#Person2#")
row['summary'] = row['summary'].str.replace("#PersoneName#","#Person2#")
train_df[train_df['fname']=='train_5614'] = row

row = train_df[train_df['fname']=='train_6348']
row['dialogue'] = row['dialogue'].str.replace("#Name#","#Person2#")
row['summary'] = row['summary'].str.replace("#Name#","#Person2#")
train_df[train_df['fname']=='train_6348'] = row

row = train_df[train_df['fname']=='train_6608']
row['dialogue'] = row['dialogue'].str.replace("#FlightNumber#","Flight FA2028")
row['summary'] = row['summary'].str.replace("#FlightNumber#","Flight FA2028")
train_df[train_df['fname']=='train_6608'] = row

row = train_df[train_df['fname']=='train_11369']
row['dialogue'] = row['dialogue'].str.replace("#Name#","#Person1#")
row['summary'] = row['summary'].str.replace("#Name#","#Person1#")
train_df[train_df['fname']=='train_11369'] = row

row = train_df[train_df['fname']=='train_7017']
for a,b in zip(["#Bob#","#Liliana#","#Mike#","#Mike Sanderson#"], ["Bob","Liliana","Mike","Mike Sanderson"]):
    row['dialogue'] = row['dialogue'].str.replace(a,b)
    row['summary'] = row['summary'].str.replace(a,b)
train_df[train_df['fname']=='train_7017'] = row

row = train_df[train_df['fname']=='train_6829']
for a,b in zip(["#Alex#","#Kristin#"], ["Alex","Kristin"]):
    row['dialogue'] = row['dialogue'].str.replace(a,b)
    row['summary'] = row['summary'].str.replace(a,b)
train_df[train_df['fname']=='train_6829'] = row

In [69]:
print(train_df[train_df['fname']=='train_6829']['dialogue'].values[0])

#Person1#: 자, 그럼 내년 우리의 마케팅 전략에 대한 개요를 마치겠습니다. 시간을 내주셔서 감사합니다.
#Person2#: 와, 정말 멋진 발표였어요! 솔직히 말해서, #Person1#의 전략 개요에 완전히 놀랐어요. Alex, 오늘 정말 감동 받았어요.
#Person1#: 에이, 별거 아니에요. 그냥 제 일을 한 거예요.
#Person2#: 아니에요, 여기서 인정받을 자격이 있어요. 이전 발표들을 돌이켜보면 이번엔 큰 발전이 있었어요.
#Person1#: Kristin이 슬라이드를 준비하는 데 도와줬어요. Kristin은 파워포인트의 달인입니다.
#Person2#: 그리고 가격 전략에 대한 내 피드백을 반영했더라고요. 제 제안을 고려해줘서 정말 고맙습니다.
#Person1#: 네, 정말 훌륭한 조언이었어요. 좋은 포인트들을 주셨죠.
#Person2#: 정말 멋진 일을 해냈다고 말하고 싶었어요. 정말 잘했어요.


In [71]:
train_df.to_csv("train_clean_special_tokens.csv",index=False)

In [78]:
result = train_df['summary'].apply(len)
print(result.min(), result.max(), result.mean())

13 376 85.78863289716625


In [ ]:
### Person1 으로 시작하는 데이터 개수, Person2로 시작하는 데이터 개수, 그 외 개수

starts_w_person1 = list(train_df[train_df['summary'].str.startswith("#Person1#")].index)
starts_w_person2 = list(train_df[train_df['summary'].str.startswith("#Person2#")].index)
starts_w_person = starts_w_person1 + starts_w_person2
others = list(train_df[~train_df.index.isin(starts_w_person)].index)
print(len(starts_w_person1), len(starts_w_person2), len(others))
# 357_289 11847 7169 5898
# 6026 3554 2877

starts_w_person1 = list(val_df[val_df['summary'].str.startswith("#Person1#")].index)
starts_w_person2 = list(val_df[val_df['summary'].str.startswith("#Person2#")].index)
starts_w_person = starts_w_person1 + starts_w_person2
others = list(val_df[~val_df.index.isin(starts_w_person)].index)
print(len(starts_w_person1), len(starts_w_person2), len(others))

sub_df['summary'] = sub_df['summary'].apply(str.strip)
starts_w_person1 = list(sub_df[sub_df['summary'].str.startswith("#Person1#")].index)
starts_w_person2 = list(sub_df[sub_df['summary'].str.startswith("#Person2#")].index)
starts_w_person = starts_w_person1 + starts_w_person2
others = list(sub_df[~sub_df.index.isin(starts_w_person)].index)
print(len(starts_w_person1), len(starts_w_person2), len(others))

6026 3554 2877
270 131 98
347 15 137


In [119]:
import re
def get_indices_with_regex(df: pd.DataFrame, pattern: str):
    """
    dialogue와 summary 컬럼에서 특정 정규표현식을 만족하는 샘플의 인덱스를 추출합니다.

    Args:
        df (pd.DataFrame): 'dialogue'와 'summary' 컬럼을 포함하는 pandas DataFrame.
        pattern (str): 정규표현식 패턴 문자열.

    Returns:
        List[Union[int, str]]: 정규표현식에 매칭되는 샘플의 인덱스 리스트.
    """
    if 'dialogue' not in df.columns or 'summary' not in df.columns:
        raise ValueError("DataFrame must contain 'dialogue' and 'summary' columns.")

    # 각 컬럼에서 정규표현식을 만족하는 행을 boolean Series로 반환
    # dialogue_matches = df['dialogue'].str.contains(pattern, na=False, regex=True)
    # summary_matches = df['summary'].str.contains(pattern, na=False, regex=True)

    dialogue_matches = df['dialogue'].apply(lambda x: True if re.findall(pattern,x) else False)
    summary_matches = df['summary'].apply(lambda x: True if re.findall(pattern,x) else False)    

    # 두 컬럼 중 하나라도 조건을 만족하는 행을 찾기 위해 OR 연산
    combined_matches = dialogue_matches | summary_matches

    # 조건을 만족하는 행의 인덱스를 추출하여 리스트로 반환
    indices = df[combined_matches].index.tolist()

    return indices

In [120]:
idxs = get_indices_with_regex(test_df, r"#Person1#(?:[가-힣]+)?\s*#Person1#")
print(len(idxs))
print(test_df.iloc[idxs].iloc[0]['summary'])

116
 #Person1#은 #Person1#에게 사람 중 누가 진짜 인품이 뛰어나다고 생각하는지 묻고, #Person2#는 아브라함 링컨을 떠올립니다. 


In [121]:
idxs = get_indices_with_regex(test_df, r"#Person2#(?:[가-힣]+)?\s*#Person2#")
print(len(idxs))
print(test_df.iloc[idxs].iloc[0]['summary'])

6
 #Person2#은 #Person2#에게 삶을 어떻게 조정할지 조언을합니다. #Person1#는 매일 밤 잠을자고, 와인을많이 마시고,매일아침 운동을하며,미래를 걱정합니다 
